In [1]:
import pandas as pd
df = pd.read_excel('ADNI combined.xlsx', sheet_name='sample')
#print(df)
sid = df['RID']
grp = df['Group at scan date (1=CN, 2=EMCI, 3=LMCI, 4=AD, 5=SMC)']
age = df['Age at scan']
sex = df['Sex (1=female)']
tiv = df['TIV']
field = df['MRI_Field_Strength']
grpbin = (grp > 1) # 1=CN, ...


In [2]:
import glob
dataAD = sorted(glob.glob('mwp1_MNI/AD/*.nii.gz'))
dataLMCI = sorted(glob.glob('mwp1_MNI/LMCI/*.nii.gz'))
dataCN = sorted(glob.glob('mwp1_MNI/CN/*.nii.gz'))
#dataADNI3 = sorted(glob.glob('mwp1_MNI/ADNI3/*.nii.gz'))
dataFiles = dataAD + dataLMCI + dataCN # + dataADNI3
numfiles = len(dataFiles)
print('Found ', str(numfiles), ' nifti files')


Found  663  nifti files


In [3]:
# Match covariate information
import re
debug = False
cov_idx = [-1] * numfiles # list; array: np.full((numfiles, 1), -1, dtype=int)
print('Matching covariates for loaded files ...')
for i,id in enumerate(sid):
  p = [j for j,x in enumerate(dataFiles) if re.search('_%04d_' % id, x)] # translate ID numbers to four-digit numbers, get both index and filename
  if len(p)==0:
    if debug: print('Did not find %04d' % id) # did not find Excel sheet subject ID in loaded file selection
  else:
    if debug: print('Found %04d in %s: %s' % (id, p[0], dataFiles[p[0]]))
    cov_idx[p[0]] = i # store Excel index i for data file index p[0]
print('Checking for scans not found in Excel sheet: ', sum(x<0 for x in cov_idx))

labels = pd.DataFrame({'Group':grpbin}).iloc[cov_idx, :]
grps = pd.DataFrame({'Group':grp, 'RID':sid}).iloc[cov_idx, :]

Matching covariates for loaded files ...
Checking for scans not found in Excel sheet:  0


In [4]:
# Load residualized data from disk
import h5py
import numpy as np
from pandas import DataFrame
from keras.utils import to_categorical
hf = h5py.File('residuals_wb_mwp1_MNI.hdf5', 'r')
hf.keys # read keys
labels = np.array(hf.get('labels')) # note: was of data frame type before
images = np.array(hf.get('images'))
hf.close()
labels = to_categorical(np.asarray(labels)) # use grps to access original labels
print(images.shape)

Using TensorFlow backend.


(663, 100, 100, 120, 1)


In [5]:
# specify version of tensorflow
#%tensorflow_version 1.x
#%tensorflow_version 2.x
import tensorflow as tf
print(tf.__version__)
# disable tensorflow deprecation warnings
import logging
logging.getLogger('tensorflow').disabled=True
# downgrade to specific version
#!pip install tensorflow-gpu==1.15
#import tensorflow as tf
#print(tf.__version__)
from keras.backend.tensorflow_backend import set_session
config = tf.ConfigProto(
    gpu_options = tf.GPUOptions(per_process_gpu_memory_fraction=0.8)
    # device_count = {'GPU': 1}
)
config.gpu_options.allow_growth = False #True
session = tf.Session(config=config)
set_session(session)

1.15.0


In [6]:
# add new column of prediction value
grps['pred'] = -1.0
grps['predclass'] = -1.0
test_indices = []

In [7]:
def get_values(conf_matrix):
    assert conf_matrix.shape==(2,2)
    tn, fp, fn, tp = conf_matrix.ravel()
    sen = tp / (tp+fn)
    spec = tn / (fp+tn)
    ppv = tp / (tp+fp)
    npv = tn / (tn+fn)
    f1 = 2 * ((sen * spec) / (sen + spec))
    return sen, spec, ppv, npv, f1

In [21]:
# Split data into training/validation and holdout test data
from sklearn.model_selection import StratifiedKFold,train_test_split
import numpy as np
import gc
from sklearn.metrics import roc_curve, auc
from matplotlib import pyplot as plt
#%matplotlib inline
import os
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"]="0" # model will be trained on GPU 0

import keras
from keras import layers
from keras.layers.normalization import BatchNormalization
from keras import models
from keras.optimizers import Adam
from keras import initializers
from keras.callbacks import EarlyStopping
import tensorflow as tf
from statistics import mean,stdev
from sklearn.metrics import confusion_matrix

acc_AD, acc_MCI, auc_AD, auc_MCI = [], [], [], []
opt_acc_AD, opt_acc_MCI = [], []
sen_AD, sen_MCI, spec_AD, spec_MCI = [], [], [], []
ppv_AD, ppv_MCI, npv_AD, npv_MCI = [], [], [], []
opt_ppv_AD, opt_ppv_MCI, opt_npv_AD, opt_npv_MCI = [], [], [], []
opt_sen_AD, opt_sen_MCI, opt_spec_AD, opt_spec_MCI = [], [], [], []
f1_AD, f1_MCI, opt_f1_AD, opt_f1_MCI = [], [], [], []
skf = StratifiedKFold(n_splits=20, shuffle=True, random_state=42)
batch_size = 8
k = 1
for train_idX, test_idX in skf.split(X=images, y=grps.iloc[:, 0]): # split data as tenfold stratified cross-validation
    gc.collect() # run garbage collector
    print('Distribution of diagnoses in training data: [1=CN, 3=LMCI, 4=AD]')
    print(grps.iloc[train_idX, :].Group.value_counts())
    testgrps = grps.iloc[test_idX, :]
    print(testgrps.RID) # prints test RID
    print('Distribution of diagnoses in holdout test data: [1=CN, 3=LMCI, 4=AD]')
    print(testgrps.Group.value_counts())
    
    test_indices.append(test_idX)
    testdat = images[test_idX, :]
    test_Y = labels[test_idX, :]

    print('validating model newmodel/newmodel_wb_cv%d.hdf5' % k)
    mymodel = models.load_model('newmodel/newmodel_wb_cv%d.hdf5' % k)
    k = k+1

    pred = mymodel.predict(testdat, batch_size=batch_size)
    grps.iloc[test_idX, 2] = pred[:, 1]
    fpr = dict()
    tpr = dict()
    roc_auc = dict()
    acc = dict()
    opt_acc = dict()
    thresh = dict()

    # redo AUC for binary comparison: AD vs. HC and MCI vs. HC
    for i in [3,4]:
      grpi = np.equal(testgrps.Group.to_numpy(dtype=np.int), np.ones((testgrps.shape[0],), dtype=np.int)*i)
      grp1 = np.equal(testgrps.Group.to_numpy(dtype=np.int), np.ones((testgrps.shape[0],), dtype=np.int))
      grpidx = np.logical_or(grpi, grp1)
      fpr[i], tpr[i], thresholds = roc_curve(test_Y[grpidx, 1], pred[grpidx, 1])
      thresh[i] = thresholds[np.argmax(tpr[i] - fpr[i])]
      print(thresh[i])
      roc_auc[i] = auc(fpr[i], tpr[i])
      acc[i] = np.mean((test_Y[grpidx, 1] == np.round(pred[grpidx, 1])).astype(int))*100
      opt_acc[i] = np.mean((test_Y[grpidx, 1] == (pred[grpidx, 1]>thresh[i]).astype(int)).astype(int))*100

    auc_AD.append(roc_auc[4])
    auc_MCI.append(roc_auc[3])
    acc_AD.append(acc[4])
    acc_MCI.append(acc[3])
    opt_acc_AD.append(opt_acc[4])
    opt_acc_MCI.append(opt_acc[3])
    
    print('AUC for MCI vs. CN = %0.3f' % roc_auc[3])
    print('AUC for AD vs. CN = %0.3f' % roc_auc[4])
    print('Acc for MCI vs. CN = %0.1f' % acc[3])
    print('Acc for AD vs. CN = %0.1f' % acc[4])
    print('confusion matrix (naive)')
    confmat = confusion_matrix(testgrps.Group-1, np.round(pred[:, 1]))
    sen, spec, ppv, npv, f1 = get_values(confmat[(0,2),0:2]) # MCI
    sen_MCI.append(sen); spec_MCI.append(spec); ppv_MCI.append(ppv); npv_MCI.append(npv); f1_MCI.append(f1)
    sen, spec, ppv, npv, f1 = get_values(confmat[(0,3),0:2]) # AD
    sen_AD.append(sen); spec_AD.append(spec); ppv_AD.append(ppv); npv_AD.append(npv); f1_AD.append(f1)
    print(confmat[:,(0,1)])
    print('oAcc for MCI vs. CN = %0.1f' % opt_acc[3])
    print('oAcc for AD vs. CN = %0.1f' % opt_acc[4])
    print('confusion matrix (optimized)')
    confmat = confusion_matrix(testgrps.Group-1, (pred[:, 1]>thresh[3]).astype(int)) ## use MCI threshold
    sen, spec, ppv, npv, f1 = get_values(confmat[(0,2),0:2]) # MCI
    opt_sen_MCI.append(sen); opt_spec_MCI.append(spec); opt_ppv_MCI.append(ppv); opt_npv_MCI.append(npv); opt_f1_MCI.append(f1)
    sen, spec, ppv, npv, f1 = get_values(confmat[(0,3),0:2]) # AD
    opt_sen_AD.append(sen); opt_spec_AD.append(spec); opt_ppv_AD.append(ppv); opt_npv_AD.append(npv); opt_f1_AD.append(f1)
    print(confmat[:,(0,1)])
    # break

Distribution of diagnoses in training data: [1=CN, 3=LMCI, 4=AD]
1    241
3    209
4    179
Name: Group, dtype: int64
1364    1030
1600    5241
1307     552
1315     667
1280     230
1473    4990
238     4201
528     4707
574     4801
621     4894
1436    4251
1320     698
1459    4899
1371    1072
1376    1118
1304     505
1446    4542
1332     800
1641    6180
174     4079
640     4925
1310     610
1329     751
1327     741
4        413
209     4150
335     4384
362     4421
368     4429
381     4449
407     4496
415     4506
458     4580
478     4609
Name: RID, dtype: int64
Distribution of diagnoses in holdout test data: [1=CN, 3=LMCI, 4=AD]
1    13
3    11
4    10
Name: Group, dtype: int64
validating model newmodel/newmodel_wb_cv1.hdf5
0.9997429
0.99872404
AUC for MCI vs. CN = 0.692
AUC for AD vs. CN = 0.992
Acc for MCI vs. CN = 66.7
Acc for AD vs. CN = 78.3
confusion matrix (naive)
[[ 8  5]
 [ 0  0]
 [ 3  8]
 [ 0 10]]
oAcc for MCI vs. CN = 70.8
oAcc for AD vs. CN = 91.3
confusion 

0.06172751
0.96967936
AUC for MCI vs. CN = 0.804
AUC for AD vs. CN = 0.808
Acc for MCI vs. CN = 70.8
Acc for AD vs. CN = 73.9
confusion matrix (naive)
[[10  3]
 [ 0  0]
 [ 4  7]
 [ 3  7]]
oAcc for MCI vs. CN = 70.8
oAcc for AD vs. CN = 82.6
confusion matrix (optimized)
[[8 5]
 [0 0]
 [2 9]
 [3 7]]
Distribution of diagnoses in training data: [1=CN, 3=LMCI, 4=AD]
1    241
3    209
4    180
Name: Group, dtype: int64
1321     702
1361    1007
1456    4859
1636    6039
1278     227
181     4089
211     4152
243     4215
552     4755
1417    4115
1451    4729
1398    1380
1419    4167
203     4138
246     4219
286     4294
296     4324
318     4359
592     4842
657     4976
1430    4213
1344     907
1271     171
2        295
175     4080
210     4151
274     4278
317     4357
336     4385
366     4427
388     4464
414     4505
648     4951
Name: RID, dtype: int64
Distribution of diagnoses in holdout test data: [1=CN, 3=LMCI, 4=AD]
1    13
3    11
4     9
Name: Group, dtype: int64
validating 

0.24965087
0.9999906
AUC for MCI vs. CN = 0.720
AUC for AD vs. CN = 0.954
Acc for MCI vs. CN = 65.2
Acc for AD vs. CN = 76.2
confusion matrix (naive)
[[8 4]
 [0 0]
 [4 7]
 [1 8]]
oAcc for MCI vs. CN = 69.6
oAcc for AD vs. CN = 85.7
confusion matrix (optimized)
[[7 5]
 [0 0]
 [2 9]
 [0 9]]
Distribution of diagnoses in training data: [1=CN, 3=LMCI, 4=AD]
1    242
3    209
4    180
Name: Group, dtype: int64
1325     729
1369    1057
1568    5187
1645    6303
1606    5251
1488    5028
1542    5138
327     4373
654     4962
1649    6427
1441    4302
1328     746
345     4394
376     4444
418     4510
422     4515
433     4538
476     4605
597     4852
622     4896
1251      61
1248      56
1342     896
1259     113
141     4020
245     4218
311     4350
321     4365
374     4441
382     4453
413     4503
593     4843
Name: RID, dtype: int64
Distribution of diagnoses in holdout test data: [1=CN, 3=LMCI, 4=AD]
1    12
3    11
4     9
Name: Group, dtype: int64
validating model newmodel/newmode

In [22]:
# print model performance summary
print('Mean Acc for MCI vs. CN = %0.1f +/- %0.1f' % (mean(acc_MCI), stdev(acc_MCI)))
print('Mean Acc for AD vs. CN = %0.1f +/- %0.1f' % (mean(acc_AD), stdev(acc_AD)))
print('Mean oAcc for MCI vs. CN = %0.1f +/- %0.1f' % (mean(opt_acc_MCI), stdev(opt_acc_MCI)))
print('Mean oAcc for AD vs. CN = %0.1f +/- %0.1f' % (mean(opt_acc_AD), stdev(opt_acc_AD)))
print('Mean AUC for MCI vs. CN = %0.3f +/- %0.3f' % (mean(auc_MCI), stdev(auc_MCI)))
print('Mean AUC for AD vs. CN = %0.3f +/- %0.3f' % (mean(auc_AD), stdev(auc_AD)))

print('results details:')
results = pd.DataFrame({'Accuracy_MCI':acc_MCI, 'Accuracy_AD':acc_AD, 'AUC_MCI':auc_MCI, 'AUC_AD':auc_AD, 
                        'Opt_acc_MCI':opt_acc_MCI, 'Opt_acc_AD':opt_acc_AD})
print(results)

fpr_all, tpr_all, thresholds_all = roc_curve(labels[:, 1], grps.pred)
thresh_all = thresholds_all[np.argmax(tpr_all - fpr_all)]
print(thresh_all)
roc_auc_all = auc(fpr_all, tpr_all)
acc_all = np.mean((labels[:, 1] == np.round(grps.pred)).astype(int))*100
opt_acc_all = np.mean((labels[:, 1] == (grps.pred>thresh_all).astype(int)).astype(int))*100

confmat = confusion_matrix(grps.Group-1, (grps.pred>thresh_all).astype(int))
print('Naive accuracy: %0.1f %%' % acc_all)
print('confusion matrix (naive)')
print(confusion_matrix(grps.Group-1, np.round(grps.pred))[:,(0,1)])

print('Optimized accuracy: %0.1f %%' % opt_acc_all)
print('confusion matrix (optimized)')
print(confmat[:,(0,1)])

Mean Acc for MCI vs. CN = 69.2 +/- 9.4
Mean Acc for AD vs. CN = 77.3 +/- 6.8
Mean oAcc for MCI vs. CN = 73.6 +/- 7.8
Mean oAcc for AD vs. CN = 87.8 +/- 5.8
Mean AUC for MCI vs. CN = 0.773 +/- 0.103
Mean AUC for AD vs. CN = 0.943 +/- 0.056
results details:
    Accuracy_MCI  Accuracy_AD   AUC_MCI    AUC_AD  Opt_acc_MCI  Opt_acc_AD
0      66.666667    78.260870  0.692308  0.992308    70.833333   91.304348
1      66.666667    78.260870  0.699301  1.000000    62.500000   95.652174
2      75.000000    78.260870  0.825175  1.000000    79.166667   95.652174
3      70.833333    65.217391  0.881119  0.861538    87.500000   82.608696
4      66.666667    73.913043  0.783217  0.930769    70.833333   86.956522
5      62.500000    73.913043  0.811189  0.946154    75.000000   82.608696
6      75.000000    73.913043  0.755245  0.892308    70.833333   78.260870
7      79.166667    78.260870  0.916084  0.976923    79.166667   91.304348
8      70.833333    73.913043  0.804196  0.807692    70.833333   82.6

In [25]:
print('Mean Sen for MCI vs. CN = %0.3f +/- %0.3f' % (mean(sen_MCI), stdev(sen_MCI)))
print('Mean Sen for AD vs. CN = %0.3f +/- %0.3f' % (mean(sen_AD), stdev(sen_AD)))
print('Mean oSen for MCI vs. CN = %0.3f +/- %0.3f' % (mean(opt_sen_MCI), stdev(opt_sen_MCI)))
print('Mean oSen for AD vs. CN = %0.3f +/- %0.3f' % (mean(opt_sen_AD), stdev(opt_sen_AD)))

print('Mean Spec for MCI vs. CN = %0.3f +/- %0.3f' % (mean(spec_MCI), stdev(spec_MCI)))
print('Mean Spec for AD vs. CN = %0.3f +/- %0.3f' % (mean(spec_AD), stdev(spec_AD)))
print('Mean oSpec for MCI vs. CN = %0.3f +/- %0.3f' % (mean(opt_spec_MCI), stdev(opt_spec_MCI)))
print('Mean oSpec for AD vs. CN = %0.3f +/- %0.3f' % (mean(opt_spec_AD), stdev(opt_spec_AD)))

print('Mean PPV for MCI vs. CN = %0.3f +/- %0.3f' % (mean(ppv_MCI), stdev(ppv_MCI)))
print('Mean PPV for AD vs. CN = %0.3f +/- %0.3f' % (mean(ppv_AD), stdev(ppv_AD)))
print('Mean oPPV for MCI vs. CN = %0.3f +/- %0.3f' % (mean(opt_ppv_MCI), stdev(opt_ppv_MCI)))
print('Mean oPPV for AD vs. CN = %0.3f +/- %0.3f' % (mean(opt_ppv_AD), stdev(opt_ppv_AD)))

print('Mean NPV for MCI vs. CN = %0.3f +/- %0.3f' % (mean(npv_MCI), stdev(npv_MCI)))
print('Mean NPV for AD vs. CN = %0.3f +/- %0.3f' % (mean(npv_AD), stdev(npv_AD)))
print('Mean oNPV for MCI vs. CN = %0.3f +/- %0.3f' % (mean(opt_npv_MCI), stdev(opt_npv_MCI)))
print('Mean oNPV for AD vs. CN = %0.3f +/- %0.3f' % (mean(opt_npv_AD), stdev(opt_npv_AD)))

print('Mean F1 for MCI vs. CN = %0.3f +/- %0.3f' % (mean(f1_MCI), stdev(f1_MCI)))
print('Mean F1 for AD vs. CN = %0.3f +/- %0.3f' % (mean(f1_AD), stdev(f1_AD)))
print('Mean oF1 for MCI vs. CN = %0.3f +/- %0.3f' % (mean(opt_f1_MCI), stdev(opt_f1_MCI)))
print('Mean oF1 for AD vs. CN = %0.3f +/- %0.3f' % (mean(opt_f1_AD), stdev(opt_f1_AD)))

Mean Sen for MCI vs. CN = 0.732 +/- 0.181
Mean Sen for AD vs. CN = 0.926 +/- 0.095
Mean oSen for MCI vs. CN = 0.645 +/- 0.128
Mean oSen for AD vs. CN = 0.876 +/- 0.119
Mean Spec for MCI vs. CN = 0.658 +/- 0.124
Mean Spec for AD vs. CN = 0.658 +/- 0.124
Mean oSpec for MCI vs. CN = 0.814 +/- 0.178
Mean oSpec for AD vs. CN = 0.814 +/- 0.178
Mean PPV for MCI vs. CN = 0.652 +/- 0.121
Mean PPV for AD vs. CN = 0.679 +/- 0.088
Mean oPPV for MCI vs. CN = 0.797 +/- 0.156
Mean oPPV for AD vs. CN = 0.810 +/- 0.152
Mean NPV for MCI vs. CN = 0.757 +/- 0.117
Mean NPV for AD vs. CN = 0.935 +/- 0.083
Mean oNPV for MCI vs. CN = 0.729 +/- 0.058
Mean oNPV for AD vs. CN = 0.906 +/- 0.087
Mean F1 for MCI vs. CN = 0.671 +/- 0.112
Mean F1 for AD vs. CN = 0.759 +/- 0.074
Mean oF1 for MCI vs. CN = 0.695 +/- 0.089
Mean oF1 for AD vs. CN = 0.826 +/- 0.108


In [26]:
print(test_indices)
print()
import json

# taken from https://stackoverflow.com/a/47626762
class NumpyEncoder(json.JSONEncoder):
    def default(self, obj):
        if isinstance(obj, np.ndarray):
            return obj.tolist()
        return json.JSONEncoder.default(self, obj)
print(json.dumps(test_indices, cls=NumpyEncoder))

[array([ 10,  31,  47,  69,  86,  90, 115, 150, 164, 174, 191, 198, 222,
       228, 244, 258, 260, 264, 271, 288, 401, 415, 418, 439, 488, 526,
       576, 589, 594, 599, 610, 614, 629, 637]), array([ 10,  31,  47,  69,  86,  90, 115, 150, 164, 174, 191, 198, 222,
       228, 244, 258, 260, 264, 271, 288, 401, 415, 418, 439, 488, 526,
       576, 589, 594, 599, 610, 614, 629, 637]), array([ 14,  50,  53,  55,  76, 130, 145, 156, 173, 184, 189, 217, 247,
       265, 267, 268, 295, 310, 338, 358, 362, 424, 428, 433, 442, 477,
       505, 529, 533, 545, 561, 607, 625, 642]), array([  2,  19,  40,  87,  96, 103, 109, 125, 152, 153, 200, 219, 243,
       279, 284, 286, 291, 329, 374, 393, 399, 419, 427, 434, 484, 495,
       506, 557, 573, 593, 618, 630, 651, 660]), array([  5,  25,  30,  67, 120, 141, 161, 167, 176, 186, 207, 229, 257,
       261, 266, 296, 313, 325, 331, 346, 357, 425, 447, 454, 475, 476,
       546, 585, 586, 590, 604, 609, 615, 646]), array([ 21,  44,  59,  62,  91, 10